##**Script for Calculating and Extracting Zonal Statistics for Fire Predictors**

**Notes regarding indices to calculate:**

*   **Fire number**: Number of times a site burned in between 1984 and the the year the plot was sampled. -- Done
*   **Fire return interval**: Average number of years between fires from 1984 and the year the plot was sampled. -- inverse of fire frequency—Not using this because unburned sites return undefined values (dividing by 0). Fire frequency is probably a better metric
*   **Fire frequency:** Number of fires that burned on a site over the period from 1984 the year the plot was sampled. -- As percentage of years with fire—done
*   **Years since most recent fire** -- Done
*   **Severity of most recent fire:** Mode value of severity class for the most recent fire prior to the year the plot was sampled. -- Done
*   **Severity of most severe fire:** Maximum mode value for each plot during the period of 1984 to the year the plot was sampled. -- Done

### **Setup Script**

**Import common python libs**

In [1]:
import os
import sys
import datetime
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
import math
import csv
import numpy as np                  # to create a sequence for plotting
from scipy.spatial import distance  # for Jensen-Shannon
import matplotlib.pyplot as plt     # for plotting histograms
import pandas as pd                 # for creating histogram dataframe to export to GDrive
from google.colab import drive      # for exporting from distributed machine to GDrive
from google.colab import files

**Install, import, & authenticate earthengine python API**

In [2]:
##@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=kctKe7Mrhn0q74ZhDwhkRjbA44HFUMPMe_t_hjWing8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4wEwzCgvakan6lHYvwJBZd4Qc3I_KatRWgVdlYlHIdih8IuSbx-o2-g

Successfully saved authorization token.
{'t

###**Import and Test Folium ImageViz** 

#### Import data to test visualization

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


####Static Map <BR>
ee.Image objects can be displayed to notebook output cells. The IPython.display module contains the Image function, which can display the results of a URL representing an image generated from a call to the Earth Engine getThumbUrl function. The following cell will display a thumbnail of the global elevation model.

In [ ]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

#### Interactive Map <BR>
The [folium](https://python-visualization.github.io/folium/) library can be used to display ee.Image objects on an interactive [Leaflet](https://leafletjs.com/) map. Folium has no default method for handling tiles from Earth Engine, so one must be defined and added to the folium.Map module before use.

The following cells provide an example of adding a method for handing Earth Engine tiles and using it to display an elevation model to a Leaflet map.

In [ ]:
# Import the folium library.
import folium
from folium import plugins

#### Add custom basemaps

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

#### Define add_ee_layer() function


In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#### Display ee.Image tiles

In [ ]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

### **Import and Process Fire Data**

**Import Geometries for Great Basin and AIM plots (with buffers) and dNBR and RdNBR severity images**

In [3]:
# ---- Import project geometries ----
GBbounds = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/GBbounds')
GBbb = ee.Feature(GBbounds.first().geometry().bounds())

# CHANGE GEOMETRY DEPENDING ON WHAT YOU'RE EXPORTING
# GBbb = ee.FeatureCollection('users/ericjensen41_default/Thesis/Project_Boundaries/SaddleDraw').first().geometry()

# ---- Import AIM plots ----
AIMplots = ee.FeatureCollection('users/ericjensen41_default/Thesis/Plots/Allplots')

# Buffer function to apply to plots
def buffer100(f):
  return f.buffer(100)

AIMplots = AIMplots.map(buffer100)

print(AIMplots.first().getInfo())
print(AIMplots.size().getInfo())
# print(GBbounds.getInfo())

# ---- Import dNBR and RdNBR rasters ----
# dnbr = ee.Image('users/ericjensen41_default/Thesis/Covariates/dNBR_GB_1984_2017')
# rdnbr = ee.Image('users/ericjensen41_default/Thesis/Covariates/RdNBR_GB_1984_2017')

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-120.12868625825102, 39.81919187231709], [-120.12901350776406, 39.81915605637096], [-120.12931470725938, 39.819051459588714], [-120.1295658805991, 39.81888640816144], [-120.12974703423127, 39.818674040560715], [-120.12984374872453, 39.81843126159827], [-120.12984832643083, 39.81817739669358], [-120.12976040396045, 39.81793265350364], [-120.12958698079721, 39.81771651339021], [-120.12934186187259, 39.81754618076458], [-120.12904455855359, 39.81743521372314], [-120.12871873556884, 39.81739244494162], [-120.12839032748839, 39.817421278687995], [-120.12808547462511, 39.81751941988239], [-120.1278284425685, 39.81767905675846], [-120.12763969086173, 39.81788748259494], [-120.12753424448718, 39.8181281070539], [-120.12752049776961, 39.81838177665972], [-120.12759954594036, 39.81862829934358], [-120.1277650976473, 39.81884805172217], [-120.12800397547097, 39.8190235411745], [-120.1282971646923, 39.819140798358], [-120.128621

In [4]:
# ---- Import dNBR and RdNBR rasters ----
d84 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1984dNBR').rename('dnbr_1984')
d85 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1985dNBR').rename('dnbr_1985')
d86 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1986dNBR').rename('dnbr_1986')
d87 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1987dNBR').rename('dnbr_1987')
d88 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1988dNBR').rename('dnbr_1988')
d89 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1989dNBR').rename('dnbr_1989')
d90 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1990dNBR').rename('dnbr_1990')
d91 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1991dNBR').rename('dnbr_1991')
d92 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1992dNBR').rename('dnbr_1992')
d93 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1993dNBR').rename('dnbr_1993')
d94 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1994dNBR').rename('dnbr_1994')
d95 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1995dNBR').rename('dnbr_1995')
d96 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1996dNBR').rename('dnbr_1996')
d97 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1997dNBR').rename('dnbr_1997')
d98 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1998dNBR').rename('dnbr_1998')
d99 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1999dNBR').rename('dnbr_1999')
d00 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2000dNBR').rename('dnbr_2000')
d01 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2001dNBR').rename('dnbr_2001')
d02 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2002dNBR').rename('dnbr_2002')
d03 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2003dNBR').rename('dnbr_2003')
d04 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2004dNBR').rename('dnbr_2004')
d05 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2005dNBR').rename('dnbr_2005')
d06 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2006dNBR').rename('dnbr_2006')
d07 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2007dNBR').rename('dnbr_2007')
d08 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2008dNBR').rename('dnbr_2008')
d09 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2009dNBR').rename('dnbr_2009')
d10 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2010dNBR').rename('dnbr_2010')
d11 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2011dNBR').rename('dnbr_2011')
d12 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2012dNBR').rename('dnbr_2012')
d13 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2013dNBR').rename('dnbr_2013')
d14 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2014dNBR').rename('dnbr_2014')
d15 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2015dNBR').rename('dnbr_2015')
d16 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2016dNBR').rename('dnbr_2016')
d17 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2017dNBR').rename('dnbr_2017')
r84 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1984RdNBR').rename('rdnbr_1984')
r85 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1985RdNBR').rename('rdnbr_1985')
r86 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1986RdNBR').rename('rdnbr_1986')
r87 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1987RdNBR').rename('rdnbr_1987')
r88 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1988RdNBR').rename('rdnbr_1988')
r89 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1989RdNBR').rename('rdnbr_1989')
r90 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1990RdNBR').rename('rdnbr_1990')
r91 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1991RdNBR').rename('rdnbr_1991')
r92 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1992RdNBR').rename('rdnbr_1992')
r93 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1993RdNBR').rename('rdnbr_1993')
r94 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1994RdNBR').rename('rdnbr_1994')
r95 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1995RdNBR').rename('rdnbr_1995')
r96 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1996RdNBR').rename('rdnbr_1996')
r97 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1997RdNBR').rename('rdnbr_1997')
r98 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1998RdNBR').rename('rdnbr_1998')
r99 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/1999RdNBR').rename('rdnbr_1999')
r00 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2000RdNBR').rename('rdnbr_2000')
r01 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2001RdNBR').rename('rdnbr_2001')
r02 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2002RdNBR').rename('rdnbr_2002')
r03 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2003RdNBR').rename('rdnbr_2003')
r04 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2004RdNBR').rename('rdnbr_2004')
r05 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2005RdNBR').rename('rdnbr_2005')
r06 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2006RdNBR').rename('rdnbr_2006')
r07 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2007RdNBR').rename('rdnbr_2007')
r08 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2008RdNBR').rename('rdnbr_2008')
r09 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2009RdNBR').rename('rdnbr_2009')
r10 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2010RdNBR').rename('rdnbr_2010')
r11 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2011RdNBR').rename('rdnbr_2011')
r12 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2012RdNBR').rename('rdnbr_2012')
r13 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2013RdNBR').rename('rdnbr_2013')
r14 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2014RdNBR').rename('rdnbr_2014')
r15 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2015RdNBR').rename('rdnbr_2015')
r16 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2016RdNBR').rename('rdnbr_2016')
r17 = ee.Image('users/ericjensen41_default/Thesis/Covariates/MTBS/2017RdNBR').rename('rdnbr_2017')

rdnbr = r84.addBands([r85,r86,r87,r88,r89,r90,r91,r92,r93,r94,r95,r96,r97,r98,r99,r00,r01,r02,
                        r03,r04,r05,r06,r07,r08,r09,r10,r11,r12,r13,r14,r15,r16,r17])
                        
dnbr = d84.addBands([d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d00,d01,d02,
                        d03,d04,d05,d06,d07,d08,d09,d10,d11,d12,d13,d14,d15,d16,d17])

dNBR_bands = dnbr.bandNames().getInfo()
print(dNBR_bands)
RdNBR_bands = rdnbr.bandNames().getInfo()
print(RdNBR_bands)

['dnbr_1984', 'dnbr_1984_1', 'dnbr_1985', 'dnbr_1986', 'dnbr_1987', 'dnbr_1988', 'dnbr_1989', 'dnbr_1990', 'dnbr_1991', 'dnbr_1992', 'dnbr_1993', 'dnbr_1994', 'dnbr_1995', 'dnbr_1996', 'dnbr_1997', 'dnbr_1998', 'dnbr_1999', 'dnbr_2000', 'dnbr_2001', 'dnbr_2002', 'dnbr_2003', 'dnbr_2004', 'dnbr_2005', 'dnbr_2006', 'dnbr_2007', 'dnbr_2008', 'dnbr_2009', 'dnbr_2010', 'dnbr_2011', 'dnbr_2012', 'dnbr_2013', 'dnbr_2014', 'dnbr_2015', 'dnbr_2016', 'dnbr_2017']
['rdnbr_1984', 'rdnbr_1985', 'rdnbr_1986', 'rdnbr_1987', 'rdnbr_1988', 'rdnbr_1989', 'rdnbr_1990', 'rdnbr_1991', 'rdnbr_1992', 'rdnbr_1993', 'rdnbr_1994', 'rdnbr_1995', 'rdnbr_1996', 'rdnbr_1997', 'rdnbr_1998', 'rdnbr_1999', 'rdnbr_2000', 'rdnbr_2001', 'rdnbr_2002', 'rdnbr_2003', 'rdnbr_2004', 'rdnbr_2005', 'rdnbr_2006', 'rdnbr_2007', 'rdnbr_2008', 'rdnbr_2009', 'rdnbr_2010', 'rdnbr_2011', 'rdnbr_2012', 'rdnbr_2013', 'rdnbr_2014', 'rdnbr_2015', 'rdnbr_2016', 'rdnbr_2017']


**Export dNBR for 2015 for Soda Fire modeling**

In [ ]:
# # Reproject and resample image by nearest neighbors to match Landsat
# LS_ref = ee.Image('users/zackrwerner/landsat_harm_reference')
# LS_proj = LS_ref.projection() # get projection of landsat harmonized image
# d14_resample = d14.resample().reproject(LS_proj).clip(GBbb) #reproject defaults to nn 

# # Export results to drive
# d14_exp_task = ee.batch.Export.image.toDrive(
#      image =  d14_resample.clip(GBbb),
#      description = 'dnbr',
#      scale = 30,
#      maxPixels = 1e13,
#      region = GBbb) 
  
# d14_exp_task.start()


# # Reproject and resample image by nearest neighbors to match Landsat
# r14_resample = r14.resample().reproject(LS_proj).clip(GBbb) #reproject defaults to nn 

# # Export results to drive
# r14_exp_task = ee.batch.Export.image.toDrive(
#      image =  r14_resample.clip(GBbb),
#      description = 'rdnbr',
#      scale = 30,
#      maxPixels = 1e13,
#      region = GBbb) 
  
# r14_exp_task.start()

**Masking pixel values that represent unburned area**<BR>


In [5]:
# Mask pixels that do not represent burned area
dnbrmask = dnbr.neq(-32768)
dnbr = dnbr.updateMask(dnbrmask)

rdnbrmask = rdnbr.neq(-32768)
rdnbr = rdnbr.updateMask(rdnbrmask)

### **Prepare dNBR and RdNBR Image Collections for calculations**

Cast raster image bands to images in image collection 

In [6]:
#Cast dNBR raster image bands to images in image collection 
d = dnbr.reproject(crs = 'EPSG:4326', crsTransform = None, scale = 30)
dnbr_IC = ee.ImageCollection([d.select(0),d.select(1),d.select(2),d.select(3),d.select(4),d.select(5),d.select(6),d.select(7),\
                                d.select(8),d.select(9),d.select(10),d.select(11),d.select(12),d.select(13),d.select(14),d.select(15),\
                                d.select(16),d.select(17),d.select(18),d.select(19),d.select(20),d.select(21),d.select(22),d.select(23),\
                                d.select(24),d.select(25),d.select(26),d.select(27),d.select(28),d.select(29),d.select(30),d.select(31),\
                                d.select(32),d.select(33)])
                              
#Cast RdNBR raster image bands to images in image collection 
R = rdnbr.reproject(crs = 'EPSG:4326', crsTransform = None, scale = 30)
rdnbr_IC = ee.ImageCollection([R.select(0),R.select(1),R.select(2),R.select(3),R.select(4),R.select(5),R.select(6),R.select(7),\
                                R.select(8),R.select(9),R.select(10),R.select(11),R.select(12),R.select(13),R.select(14),R.select(15),\
                                R.select(16),R.select(17),R.select(18),R.select(19),R.select(20),R.select(21),R.select(22),R.select(23),\
                                R.select(24),R.select(25),R.select(26),R.select(27),R.select(28),R.select(29),R.select(30),R.select(31),\
                                R.select(32),R.select(33)])

Add year property to each image collection

In [7]:
def add_year(image):
  index_str = image.get('system:index')
  index_num = ee.Number.parse(index_str);
  return(image.set({'Year': ee.Number(1984).add(index_num)} )) 

dnbr_IC = dnbr_IC.map(add_year)
rdnbr_IC = rdnbr_IC.map(add_year)

Combine image collections // this way preserves order of images by year for some reason...

In [8]:
def combine_IC(image):
  year = image.get('Year')
  rdnbr_year = rdnbr_IC.filterMetadata('Year', 'equals', year).first()
  image_wrdnbr = ee.Image(image.addBands(rdnbr_year))
  return(image_wrdnbr)

fire_IC = dnbr_IC.map(combine_IC)

### **Calculate bands of interest**

**In the end, the goal is to have an image collection with images for each year of MTBS data. Each yearly image will have bands related to the variables in the list at the top of the document**

Function to generate a new band where each pixel that burned in a given year has the value of the year, all other pixels are masked

In [9]:
def const_FireYear(image):
  indvYear = ee.Image(image).select(0)
  yearStr = ee.String(indvYear.bandNames().get(0)).slice(5,9) # select year substring of bandname
  yearNum = ee.Number.parse(yearStr) # convert year substring to a number
  yearConstant = ee.Image([yearNum]).rename(ee.String('FireYear')) # Create image with each pixel having the value of the year
  yearMask = indvYear # Mask to remove remaining pixels that didn't burn during the year
  yearMasked = yearConstant.updateMask(yearMask)
  image_const = ee.Image(image.addBands(yearMasked))
  return(image_const)  

fire_IC = fire_IC.map(const_FireYear)

Function to add a new band with the number of years of MTBS data have been collected

In [10]:
def const_YearsOfMTBS(image):
  indvYear = ee.Image(image).select(0)
  yearStr = ee.String(indvYear.bandNames().get(0)).slice(5,9) # select year substring of bandname
  yearNum = ee.Number.parse(yearStr).subtract(1983) # convert year substring to a number and subtract 1983 to get years of MTBS
  yearConstant = ee.Image([yearNum]).rename(ee.String('YearsOfMTBS')).clip(GBbb) # Create image with each pixel having the value of the year
  image_const = ee.Image(image.addBands(yearConstant.clip(GBbb.geometry())))
  return(image_const)

fire_IC = fire_IC.map(const_YearsOfMTBS)  

Function to generate a new band with the number of years since the most recent fire

In [11]:
FireYear_IC = fire_IC.select('FireYear')  # Get IC of FireYears to reduce over in function below

def YearsSinceFire(image):
  year = ee.Number(image.get('Year')) # get year of image
  FireYear_IC_spec = FireYear_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter FireYear_IC by the year and convert to multiband image
  FireYear_I_spec = FireYear_IC_spec.toBands()
  LastFire = FireYear_I_spec.reduce(ee.Reducer.max()) # Most recent fire will be the maximum
  YearsSinceFire = LastFire.subtract(year).abs().rename(ee.String('YearsSinceFire')).unmask(year.subtract(1983)) # Set NA values to constant = years of MTBS
  image_YearsSinceFire = ee.Image(image.addBands(YearsSinceFire.clip(GBbb.geometry()))) # Add the new band to the image collection and clip to GB geometry
  return(image_YearsSinceFire)

fire_IC = fire_IC.map(YearsSinceFire)

Function to generate a new band with the a count of the number of fires that have occurred up through the current year

In [12]:
def FireCount(image):
  year = ee.Number(image.get('Year')) # get year of image
  FireCount_IC_spec = FireYear_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter FireYear_IC by the year and convert to multiband image
  FireCount_I_spec = FireCount_IC_spec.toBands()
  FireCount = FireCount_I_spec.reduce(ee.Reducer.count()).rename(ee.String('FireCount')).unmask(0) # Most recent fire will be the maximum // Set NA = 0
  image_FireCount = ee.Image(image.addBands(FireCount.clip(GBbb.geometry()))) # Add the new band to the image collection and clip to GB geometry
  return image_FireCount

fire_IC = fire_IC.map(FireCount)

Function to generate a new band with the a count of the fire frequency (count of fires / years of MTBS data) based on fires that have occurred up through the current year

In [13]:
def FireFreq(image):
  FireCount = image.select('FireCount')
  YearsOfMTBS = image.select('YearsOfMTBS')
  FireFreq = FireCount.divide(YearsOfMTBS).rename(ee.String('FireFreq'))
  image_FireFreq = ee.Image(image.addBands(FireFreq.clip(GBbb.geometry()))) # Add the new band to the image collection and clip to GB geometry
  return image_FireFreq
  
fire_IC = fire_IC.map(FireFreq)

Function to generate a new band with the a count of the fire return interval (years of MTBS data / count of fires) based on fires that have occurred up through the current year // **Not currently using; see note at top of document**

In [ ]:
# def FireReturn(image):
#   FireCount = image.select('FireCount')
#   YearsOfMTBS = image.select('YearsOfMTBS')
#   FireReturn = YearsOfMTBS.divide(FireCount).rename(ee.String('FireReturnInt'))
#   image_FireReturn = image.addBands(FireReturn.clip(GBbb.geometry())) # Add the new band to the image collection and clip to GB geometry
#   return image_FireReturn
  
# fire_IC = fire_IC.map(FireReturn)

Function to generate a new band with the severity of the most severe (calculated as dNBR and RdNBR) fire based on fires that have occurred up through the current year

In [14]:
def MaxSeverity(image):
  year = ee.Number(image.get('Year')) # get year of image
  # Get mutliband images of fire severity
  dnbr_IC_spec = dnbr_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter dnbr_IC by the year and convert to multiband image
  dnbr_I_spec = dnbr_IC_spec.toBands()
  rdnbr_IC_spec = rdnbr_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter rdnbr_IC by the year and convert to multiband image
  rdnbr_I_spec = rdnbr_IC_spec.toBands()
  # Reduce to highest severity pixels, fill NA values with 0, and clip to Great Basin
  dnbrMaxSev = dnbr_I_spec.reduce(ee.Reducer.max()).rename(ee.String('dnbrMaxSev')).unmask(0).clip(GBbb.geometry())  # Most severe fire will be the maximum value (dNBR)
  rdnbrMaxSev = rdnbr_I_spec.reduce(ee.Reducer.max()).rename(ee.String('rdnbrMaxSev')).unmask(0).clip(GBbb.geometry())  # Most severe fire will be the maximum value (RdNBR)
  MaxSev_I = ee.Image([dnbrMaxSev, rdnbrMaxSev])
  # Add dnbr and rdnb maximum severity bands
  image_MaxSev = ee.Image(image.addBands(MaxSev_I)) # Add the new band to the image collection and clip to GB geometry
  return image_MaxSev

fire_IC = fire_IC.map(MaxSeverity)

Function to generate a new band with the severity of the most recent (calculated as dNBR and RdNBR) fire based on fires that have occurred up through the current year

In [15]:
def RecSeverity(image):
  year = ee.Number(image.get('Year')) # get year of image
  # Get mutliband images of fire severity
  dnbr_IC_spec = dnbr_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter dnbr_IC by the year and convert to multiband image
  dnbr_I_spec = dnbr_IC_spec.toBands()
  rdnbr_IC_spec = rdnbr_IC.filterMetadata('Year', 'less_than', year.add(1)) # Filter rdnbr_IC by the year and convert to multiband image
  rdnbr_I_spec = rdnbr_IC_spec.toBands()
  # Reduce to most recent severity pixels, fill NA values with 0, and clip to Great Basin
  dnbrRecSev = dnbr_I_spec.reduce(ee.Reducer.lastNonNull()).rename(ee.String('dnbrRecSev')).unmask(0).clip(GBbb.geometry())  # Most severe fire will be the maximum value (dNBR)
  rdnbrRecSev = rdnbr_I_spec.reduce(ee.Reducer.lastNonNull()).rename(ee.String('rdnbrRecSev')).unmask(0).clip(GBbb.geometry())  # Most severe fire will be the maximum value (RdNBR)
  RecSev_I = ee.Image([dnbrRecSev, rdnbrRecSev])
  # Add dnbr and rdnb maximum severity bands
  image_RecSev = ee.Image(image.addBands(RecSev_I)) # Add the new band to the image collection and clip to GB geometry
  return image_RecSev

fire_IC = fire_IC.map(RecSeverity)

**Keep only bands that will be used as predictor variables**

In [16]:
fire_IC = fire_IC.select(['YearsSinceFire','FireCount', 'FireFreq', 'dnbrMaxSev', 'rdnbrMaxSev', 'dnbrRecSev', 'rdnbrRecSev'])
print(fire_IC.first().bandNames().getInfo())
print(fire_IC.first().propertyNames().getInfo())
print(fire_IC.first().get('Year').getInfo())

['YearsSinceFire', 'FireCount', 'FireFreq', 'dnbrMaxSev', 'rdnbrMaxSev', 'dnbrRecSev', 'rdnbrRecSev']
['Year', 'system:footprint', 'system:version', 'system:id', 'system:asset_size', 'system:index', 'system:bands', 'system:band_names']
1984


**Reduce Regions by year for the AIM plots**

In [17]:
def return_rr(img):
  # Get image year and filter AIM year to match zonal statistics for AIM plots to the correct year
  ImgYr = img.get('Year')
  AIMYr = AIMplots.filterMetadata('VisitYear', 'equals', ImgYr)

  # Run reduceRegions for the AIM year and image year
  RR = ee.FeatureCollection(img.reduceRegions(collection = AIMYr, reducer = ee.Reducer.mean(), scale = 30))

  return(RR)

In [ ]:
AIM_RR = fire_IC.map(return_rr)

# Run 2018 and 2019 off of the 2017 rasters. Will add 1 or 2 years to each YearsSinceFire variable
rr2018 = fire_IC.filterMetadata('Year', 'equals', 2017).first().reduceRegions(collection = AIMplots.filterMetadata('VisitYear', 'equals', 2018), reducer = ee.Reducer.mean(), scale = 30)
rr2019 = fire_IC.filterMetadata('Year', 'equals', 2017).first().reduceRegions(collection = AIMplots.filterMetadata('VisitYear', 'equals', 2019), reducer = ee.Reducer.mean(), scale = 30)

print(rr2018.first().getInfo())
print(rr2018.size().getInfo())
print(AIM_RR.size().getInfo())
print(AIM_RR.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-114.44823296716524, 42.841824735102755], [-114.44857576403486, 42.841788919105], [-114.44889127315916, 42.84168432218399], [-114.44915437924523, 42.84151927057414], [-114.44934413886895, 42.84130690280398], [-114.44944544762781, 42.84106412373812], [-114.44945024230653, 42.840810258828306], [-114.44935814240883, 42.84056551573315], [-114.44917648008297, 42.8403493757848], [-114.4489197161717, 42.84017904334306], [-114.44860828896653, 42.84006807644692], [-114.44826698735582, 42.84002530772677], [-114.44792297785307, 42.84005414143144], [-114.44760364248408, 42.84015228249406], [-114.44733439953048, 42.84031191918942], [-114.44713668048843, 42.84052034485264], [-114.44702622419739, 42.840760969199856], [-114.4470118239025, 42.84101463879032], [-114.44709462702384, 42.84126116156], [-114.44726804346082, 42.84148091409882], [-114.44751826984348, 42.841656403735776], [-114.44782538810017, 42.84177366107068], [-114.44816

In [ ]:
AIM_RR_flat = AIM_RR.flatten()
AIM_RR_flat = AIM_RR_flat.merge(rr2018).merge(rr2019)
print(AIM_RR_flat.first().getInfo())
print(AIM_RR_flat.size().getInfo())

KeyboardInterrupt: ignored

In [ ]:
fire_exp_task = ee.batch.Export.table.toDrive(collection = AIM_RR_flat,
                                            description = 'Fire_predictors', 
                                            fileNamePrefix = 'Fire_predictors',
                                            fileFormat = 'CSV')
fire_exp_task.start()

### **Export images to predict over**

**Create focal grids for each image to match scale of calculations in the model**

In [18]:
#Filter to year data
year_to_export = 2016

mtbs_year = fire_IC.filterMetadata('Year', 'equals', year_to_export).select(['YearsSinceFire']).first().clip(GBbounds)

# Reproject and resample image by nearest neighbors to match Landsat
LS_ref = ee.Image('users/zackrwerner/landsat_harm_reference')
LS_proj = LS_ref.projection() # get projection of landsat harmonized image
# mtbs_year_resample = mtbs_year.resample().reproject(LS_proj) #reproject defaults to nn 

mtbs_year_resample_focal = mtbs_year.reduceNeighborhood(reducer = ee.Reducer.mean(), kernel = ee.Kernel.circle(100, 'meters')).rename(['YearsSinceFire'])

In [19]:
# Get list of bandnames to export over
namelist = mtbs_year_resample_focal.bandNames().getInfo();
print(namelist,'namelist');

for i in namelist:
  # Get image
  img = mtbs_year_resample_focal.select(i).clip(GBbounds.geometry());
  
  # Export results to drive
  topo_exp_task = ee.batch.Export.image.toDrive(
      image =  img,
      description = i + '_' + str(year_to_export),
      scale = 30,
      maxPixels = 1e13,
      region = GBbounds.geometry(),
      crs = LS_proj)
  
  topo_exp_task.start()

['YearsSinceFire'] namelist


### **Visualize the data**

In [ ]:
#Filter to 2017 data
mtbs_2017 = fire_IC.filterMetadata('Year', 'equals', 2017).first()

# Create a folium map object.
my_map = folium.Map(location=[40, -116], zoom_start=9, height=800)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
#my_map.add_ee_layer(mtbs_2017.select('rdnbrMaxSev'), {'min':0, 'max':1000}, 'dnbrMaxSev')
#my_map.add_ee_layer(mtbs_2017.select('dnbrRecSev'), {'min':0, 'max':1000}, 'dnbrRecSev')
#my_map.add_ee_layer(mtbs_2017.select('rdnbrRecSev'), {'min':0, 'max':1000}, 'rdnbrRecSev')
#my_map.add_ee_layer(mtbs_2017.select('dnbrRecSev'), {'min':0, 'max':1000}, 'dnbrRecSev')
my_map.add_ee_layer(mtbs_2017.select('FireFreq'), {'min':0, 'max': .15}, 'FireFreq')
# my_map.add_ee_layer(mtbs_2017.select('YearsSinceFire'), {'min':0, 'max': 33}, 'YearsSinceFire')
#my_map.add_ee_layer(mtbs_2017.select('FireCount'), {'min':0, 'max': 7}, 'FireCount')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)